In [1]:
import numpy as np
import cv2
from numpy import load
from numpy import zeros
from numpy import ones
from numpy.random import randint
from numpy.random import randn
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import RepeatVector
from tensorflow.keras.layers import Embedding
from matplotlib import pyplot
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import load_model


import tensorflow as tf

In [2]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 11GB of memory on the first GPU
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=10240)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [3]:
# define the discriminator model
def define_discriminator(image_shape, vector_shape, n_classes=7):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    # source image input
    in_src_image = Input(shape=image_shape)
    # vector input
    in_target = Input(shape=(vector_shape[0], vector_shape[1]))
    # label input
    in_label = Input(shape=(1,))
    # Vector layer
    n_nodes = (image_shape[0] * image_shape[1] * image_shape[2])
    # embedding for categorical input
    li = Embedding(n_classes, 50)(in_label)
    # scale up to image dimensions with linear
    li = Dense(n_nodes)(li)
    # reshape to additional channel
    li = Reshape((image_shape[0], image_shape[1], image_shape[2]))(li)
    # LSTM Model
    p = LSTM(200, activation='relu')(in_target)
    p = Dense(n_nodes)(p)
    # Reshape and mergo to image dimension
    p = Reshape((image_shape[0], image_shape[1], image_shape[2]))(p)
    merged = Concatenate()([p, in_src_image, li])
    # concatenate images channel-wise
    # C64
    d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(merged)
    d = LeakyReLU(alpha=0.2)(d)
    # C128
    d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    # C256
    d = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    # C512
    d = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    # second last output layer
    d = Conv2D(512, (4,4), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    # patch output
    d = Conv2D(1, (4,4), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = Flatten()(d)
    d = Dropout(0.4)(d)
    out_layer = Dense(1, activation='sigmoid')(d)
    # define model
    model = Model([in_src_image, in_target, in_label], out_layer)
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, loss_weights=[0.5])
    return model

In [4]:
# define an encoder block
def define_encoder_block(layer_in, n_filters, batchnorm=True):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    # add downsampling layer
    g = Conv2D(n_filters, (4,4), strides=(2,2), padding='same',
    kernel_initializer=init)(layer_in)
    # conditionally add batch normalization
    if batchnorm:
        g = BatchNormalization()(g, training=True)
    # leaky relu activation
    g = LeakyReLU(alpha=0.2)(g)
    return g

In [5]:
# define the standalone generator model
def define_generator(in_shape, vector_shape, latent_dim, n_classes=7):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    # image input
    in_image = Input(shape=in_shape)
    in_lat = Input(shape=latent_dim)
    in_label = Input(shape=(1,))
    # embedding for categorical input
    li = Embedding(n_classes, 50)(in_label)
    li = Dense(latent_dim)(li)
    li = Reshape([latent_dim])(li)
    gen = LeakyReLU(alpha=0.2)(in_lat)
    # merge image gen and label input
    # encoder model
    e1 = define_encoder_block(in_image, 64, batchnorm=False)
    e2 = define_encoder_block(e1, 128)
    e3 = define_encoder_block(e2, 256)
    e4 = define_encoder_block(e3, 512)
    e5 = define_encoder_block(e4, 512)
    e6 = define_encoder_block(e5, 512)
    e7 = define_encoder_block(e6, 512)
    # bottleneck, no batch norm and relu
    b = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(e7)
    b = Activation('relu')(b)
    b = Flatten()(b)
    # Merge latent space variable input with network
    b = Concatenate()([b, gen, li])
    # Reshape to vector size
    n_nodes = (vector_shape[0] * vector_shape[1])
    b = Dense(n_nodes)(b)
    b = Reshape([vector_shape[0], vector_shape[1]])(b)
    # encoder-decoder LSTM model
    d = LSTM(200, activation='tanh')(b)
    d = RepeatVector(vector_shape[0])(d)
    d = LSTM(200, activation='tanh', return_sequences=True)(d)
    # output
    out_layer = Dense(3)(d)

    # define model input & output
    model = Model([in_image, in_lat, in_label], out_layer)
    return model

In [6]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model, image_shape, vector_shape, latent_dim):
    # make weights in the discriminator not trainable
    for layer in d_model.layers:
        if not isinstance(layer, BatchNormalization):
            layer.trainable = False
    # define the source image
    in_src = Input(shape=image_shape)
    in_lat = Input(shape=latent_dim)
    in_label = Input(shape=(1,))
    # connect the source image to the generator input
    gen_out = g_model([in_src, in_lat, in_label])
    # connect the source input and generator output to the discriminator input
    dis_out = d_model([in_src, gen_out, in_label])
    # src image as input, generated image and classification output
    model = Model([in_src, in_lat, in_label], [dis_out, gen_out])
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss=['binary_crossentropy', 'mae'], optimizer=opt, loss_weights=[1,100])
    return model

In [7]:
# load and prepare training images
def load_real_samples(filename):
	# load compressed arrays
	data = load(filename)
	# unpack arrays
	X1, X2, X3 = data['arr_0'], data['arr_1'], data['arr_2']
	return [X1, X2, X3-1]

In [8]:
# select a batch of random samples, returns images and target
def generate_real_samples(dataset, n_samples):
	# unpack dataset
	img, vec, label = dataset
	# choose random instances
	ix = randint(0, img.shape[0], n_samples)
	# retrieve selected images
	X1, X2, X3 = img[ix], vec[ix], label[ix]
	# generate 'real' class labels 
	y = ones((n_samples, 1))
	return [X1, X2, X3], y

In [9]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
    # generate points in the latent space
    x_input = randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input

In [10]:
# generate a batch of images, returns images and targets
def generate_fake_samples(g_model, samples, label, n_samples):
    # Generate latent space points
    z_input = generate_latent_points(latent_dim, n_samples)
    # generate fake instance
    X = g_model.predict([samples, z_input, label])
    # create 'fake' class labels 
    y = zeros((n_samples, 1))
    return X, y

In [11]:
# Overlay trajectories (data_y) to the image (data_x)
def create_trajectory(data_x, data_y, obs_len=10):
    # Calibration parameter to overlay for a 1280x360 resolution image
    K = np.array([[537.023764, 0, 640 , 0], 
                    [0 , 537.023764, 180, 0], 
                    [0, 0, 1, 0]])
    # Rotation matrix to obtain egocentric trajectory
    Rt = np.array([[0.028841, 0.007189, 0.999558, 1.481009],
                    [-0.999575,  0.004514,  0.028809,  0.296583],
                    [ 0.004305,  0.999964, -0.007316, -1.544537],
                    [ 0.      ,  0.      ,  0.      ,  1.      ]])

    # Resize data back to 1280x360
    data_x = cv2.resize(data_x, (1280,360))
    # Add column of ones for rotation matrix multiplication
    data_y = np.hstack((data_y, np.ones((len(data_y),1))))
    # Draw points
    for m in range(obs_len, data_y.shape[0]):
        # Rotation matrix multiplication of trajectory 
        A = np.matmul(np.linalg.inv(Rt), data_y[m, :].reshape(4, 1))
        # Egocentric view of trajectory
        B = np.matmul(K, A)
        # Circle location of trajectories 
        x = int(B[0, 0] * 1.0 / B[2, 0])
        y = int(B[1, 0] * 1.0 / B[2, 0])
        if (x < 0 or x > 1280 - 1 or y > 360 - 1):
            continue
        # Use opencv to overlay trajectories
        data_x = cv2.circle(data_x, (x, y), 3, (0, 0, 255), -1)
    return data_x

In [12]:
def generate_drvact_text(drvact):
    text = '[warnings] drvact label is not defined ...'
    if (drvact == 1):
        text = 'Go'
    elif (drvact == 2):
        text = 'Turn Left'
    elif (drvact == 3):
        text = 'Turn Right'
    elif (drvact == 4):
        text = 'U-turn'
    elif (drvact == 5):
        text = 'Left LC'
    elif (drvact == 6):
        text = 'Right LC'
    elif (drvact == 7):
        text = 'Avoidance'
    return text


In [13]:
# generate samples and save as a plot and save the model
def summarize_performance(step, g_model, dataset, n_samples=1):
    # select a sample of input images
    [X_real_img, X_real_vec, X_label], _ = generate_real_samples(dataset, n_samples)
    # generate a batch of fake samples
    X_fake_vec, _ = generate_fake_samples(g_model, X_real_img, X_label, n_samples)
    # scale all pixels from [-1,1] to [0,1]
    X_real_img = (X_real_img + 1) / 2.0
    pyplot.figure(figsize=(32.0, 20.0))
    # plot real source images
    for i in range(n_samples):
        orig_image = (X_real_img[i]* 255).astype(np.uint8)
        orig_image = cv2.resize(orig_image, (1280,360))
        pyplot.subplot(3, n_samples, 1 + i)
        pyplot.axis('off')
        pyplot.imshow(orig_image)
    # plot generated target image
    for i in range(n_samples):
        fake_sample = create_trajectory((X_real_img[i]* 255).astype(np.uint8), X_fake_vec[i])
        pyplot.subplot(3, n_samples, 1 + n_samples + i)
        pyplot.axis('off')
        pyplot.imshow(fake_sample)
    # plot real target image
    for i in range(n_samples):
        true_sample = create_trajectory((X_real_img[i]* 255).astype(np.uint8), X_real_vec[i])
        pyplot.subplot(3, n_samples, 1 + n_samples*2 + i)
        pyplot.axis('off')
        pyplot.imshow(true_sample)
    # save plot to file
    filename1 = './enc-dec-tanh-lr_0002_bs_14_epochs20/21000+plot_%06d.png' % (step+1)
    pyplot.savefig(filename1)
    pyplot.close()
    # save the generator model
    filename2 = './enc-dec-tanh-lr_0002_bs_14_epochs20/21000+model_%06d.h5' % (step+1)
    g_model.save(filename2)
    filename3 = './enc-dec-tanh-lr_0002_bs_14_epochs20/21000+d_model_%06d.h5' % (step+1)
    d_model.save(filename3)
    print('>Saved: %s, %s and %s' % (filename1, filename2, filename3))

In [14]:
# train models
def train(d_model, g_model, gan_model, dataset, latent_dim, n_epochs=20, batch_size=14):
    # iteration number to output checkpoint
    display_iter = 1000
    # calculate the number of batches
    n_batch = int(len(dataset[0]) / batch_size)
    # calculate the number of training iterations
    n_steps = n_batch * n_epochs
    # manually enumerate epochs
    for i in range(n_steps):
        # select a batch of real samples
        [X_real_img, X_real_vec, X_label], y_real = generate_real_samples(dataset, batch_size)
        # generate a batch of fake samples
        X_fake_vec, y_fake = generate_fake_samples(g_model, X_real_img, X_label, batch_size)
        # update discriminator for real samples
        d_loss1 = d_model.train_on_batch([X_real_img, X_real_vec, X_label], y_real)
        # update discriminator for generated samples
        d_loss2 = d_model.train_on_batch([X_real_img, X_fake_vec, X_label], y_fake)
        X_lat = generate_latent_points(latent_dim, batch_size)
        # update the generator
        g_loss, _, _ = gan_model.train_on_batch([X_real_img, X_lat, X_label], [y_real, X_real_vec])
        # summarize performance
        print('>%d, d1[%.3f] d2[%.3f] g[%.3f]' % (i+1, d_loss1, d_loss2, g_loss))
        # summarize model performance
        if (i+1) % int(display_iter) == 0:
            summarize_performance(i, g_model, dataset)
            print('saved')

In [15]:
%%time
# load image data
dataset = load_real_samples('./dual_condition_dataset_train.npz')
print('Loaded', dataset[0].shape, dataset[1].shape, dataset[2].shape)

Loaded (16805, 256, 256, 3) (16805, 40, 3) (16805,)
CPU times: user 22.5 s, sys: 1.06 s, total: 23.6 s
Wall time: 23.6 s


In [16]:
# define input shape based on the loaded dataset
image_shape = dataset[0].shape[1:]
vector_shape = dataset[1].shape[1:]
latent_dim = 512
# %%
# define the models
#d_model = define_discriminator(image_shape, vector_shape)
d_model = load_model('./enc-dec-tanh-lr_0002_bs_14_epochs20/d_model_021000.h5')
#g_model = define_generator(image_shape, vector_shape, latent_dim)
g_model = load_model('./enc-dec-tanh-lr_0002_bs_14_epochs20/model_021000.h5')
# %%
# define the composite model
gan_model = define_gan(g_model, d_model, image_shape, vector_shape, latent_dim)

In [17]:
# train model
train(d_model, g_model, gan_model, dataset, latent_dim)

>1, d1[0.349] d2[0.366] g[13.120]
>2, d1[0.344] d2[0.353] g[13.324]
>3, d1[0.344] d2[0.365] g[20.739]
>4, d1[0.324] d2[0.356] g[11.436]
>5, d1[0.345] d2[0.360] g[14.880]
>6, d1[0.339] d2[0.352] g[11.723]
>7, d1[0.339] d2[0.356] g[9.153]
>8, d1[0.338] d2[0.348] g[12.406]
>9, d1[0.344] d2[0.349] g[12.559]
>10, d1[0.338] d2[0.347] g[11.010]
>11, d1[0.342] d2[0.346] g[10.951]
>12, d1[0.349] d2[0.353] g[10.783]
>13, d1[0.345] d2[0.352] g[11.464]
>14, d1[0.341] d2[0.353] g[16.813]
>15, d1[0.342] d2[0.355] g[13.104]
>16, d1[0.334] d2[0.359] g[11.448]
>17, d1[0.344] d2[0.351] g[16.281]
>18, d1[0.347] d2[0.343] g[13.142]
>19, d1[0.341] d2[0.341] g[8.963]
>20, d1[0.342] d2[0.358] g[12.622]
>21, d1[0.336] d2[0.356] g[13.153]
>22, d1[0.334] d2[0.366] g[12.529]
>23, d1[0.334] d2[0.357] g[18.473]
>24, d1[0.343] d2[0.347] g[12.817]
>25, d1[0.351] d2[0.346] g[13.581]
>26, d1[0.336] d2[0.354] g[11.656]
>27, d1[0.334] d2[0.357] g[14.178]
>28, d1[0.347] d2[0.352] g[9.100]
>29, d1[0.356] d2[0.352] g[12.36

>233, d1[0.341] d2[0.362] g[9.460]
>234, d1[0.330] d2[0.354] g[10.865]
>235, d1[0.329] d2[0.354] g[12.594]
>236, d1[0.329] d2[0.368] g[10.502]
>237, d1[0.322] d2[0.369] g[10.336]
>238, d1[0.326] d2[0.367] g[10.612]
>239, d1[0.329] d2[0.378] g[10.297]
>240, d1[0.329] d2[0.370] g[7.521]
>241, d1[0.330] d2[0.372] g[14.924]
>242, d1[0.323] d2[0.377] g[14.267]
>243, d1[0.325] d2[0.358] g[11.033]
>244, d1[0.325] d2[0.372] g[15.009]
>245, d1[0.328] d2[0.363] g[12.292]
>246, d1[0.337] d2[0.371] g[12.792]
>247, d1[0.328] d2[0.373] g[12.898]
>248, d1[0.321] d2[0.370] g[14.972]
>249, d1[0.322] d2[0.367] g[11.794]
>250, d1[0.334] d2[0.372] g[15.059]
>251, d1[0.327] d2[0.367] g[13.516]
>252, d1[0.322] d2[0.375] g[14.575]
>253, d1[0.328] d2[0.371] g[13.689]
>254, d1[0.334] d2[0.361] g[11.921]
>255, d1[0.341] d2[0.372] g[13.286]
>256, d1[0.325] d2[0.371] g[12.632]
>257, d1[0.318] d2[0.375] g[9.896]
>258, d1[0.317] d2[0.366] g[11.431]
>259, d1[0.337] d2[0.378] g[11.966]
>260, d1[0.326] d2[0.356] g[18.

>462, d1[0.319] d2[0.379] g[12.975]
>463, d1[0.304] d2[0.391] g[10.750]
>464, d1[0.310] d2[0.395] g[11.691]
>465, d1[0.309] d2[0.378] g[10.624]
>466, d1[0.308] d2[0.390] g[14.092]
>467, d1[0.317] d2[0.396] g[13.542]
>468, d1[0.316] d2[0.391] g[7.796]
>469, d1[0.329] d2[0.376] g[11.989]
>470, d1[0.312] d2[0.389] g[8.681]
>471, d1[0.296] d2[0.374] g[13.888]
>472, d1[0.307] d2[0.386] g[17.986]
>473, d1[0.305] d2[0.391] g[14.048]
>474, d1[0.314] d2[0.387] g[10.028]
>475, d1[0.300] d2[0.390] g[11.337]
>476, d1[0.307] d2[0.400] g[11.523]
>477, d1[0.313] d2[0.389] g[8.930]
>478, d1[0.305] d2[0.383] g[11.984]
>479, d1[0.316] d2[0.384] g[14.680]
>480, d1[0.319] d2[0.366] g[11.590]
>481, d1[0.299] d2[0.390] g[10.501]
>482, d1[0.315] d2[0.366] g[10.498]
>483, d1[0.315] d2[0.387] g[12.896]
>484, d1[0.310] d2[0.403] g[11.926]
>485, d1[0.302] d2[0.380] g[15.177]
>486, d1[0.317] d2[0.385] g[12.624]
>487, d1[0.313] d2[0.391] g[16.538]
>488, d1[0.303] d2[0.376] g[10.207]
>489, d1[0.307] d2[0.385] g[9.6

>691, d1[0.294] d2[0.412] g[10.250]
>692, d1[0.301] d2[0.417] g[11.052]
>693, d1[0.295] d2[0.402] g[16.887]
>694, d1[0.293] d2[0.408] g[10.746]
>695, d1[0.294] d2[0.402] g[12.082]
>696, d1[0.305] d2[0.406] g[11.377]
>697, d1[0.303] d2[0.404] g[13.717]
>698, d1[0.298] d2[0.401] g[13.512]
>699, d1[0.299] d2[0.405] g[13.350]
>700, d1[0.306] d2[0.402] g[13.074]
>701, d1[0.297] d2[0.419] g[15.949]
>702, d1[0.302] d2[0.416] g[11.996]
>703, d1[0.299] d2[0.404] g[18.943]
>704, d1[0.296] d2[0.413] g[13.689]
>705, d1[0.299] d2[0.403] g[10.052]
>706, d1[0.287] d2[0.398] g[16.462]
>707, d1[0.299] d2[0.403] g[13.530]
>708, d1[0.299] d2[0.409] g[10.010]
>709, d1[0.284] d2[0.407] g[12.273]
>710, d1[0.290] d2[0.395] g[12.355]
>711, d1[0.291] d2[0.398] g[10.723]
>712, d1[0.303] d2[0.401] g[12.967]
>713, d1[0.296] d2[0.404] g[10.719]
>714, d1[0.284] d2[0.425] g[15.429]
>715, d1[0.295] d2[0.410] g[11.328]
>716, d1[0.287] d2[0.414] g[10.618]
>717, d1[0.291] d2[0.412] g[14.095]
>718, d1[0.287] d2[0.403] g[

>921, d1[0.264] d2[0.438] g[9.756]
>922, d1[0.268] d2[0.442] g[11.091]
>923, d1[0.263] d2[0.437] g[9.041]
>924, d1[0.269] d2[0.454] g[9.338]
>925, d1[0.257] d2[0.451] g[13.242]
>926, d1[0.268] d2[0.465] g[11.803]
>927, d1[0.254] d2[0.437] g[10.634]
>928, d1[0.263] d2[0.457] g[20.432]
>929, d1[0.253] d2[0.438] g[11.524]
>930, d1[0.257] d2[0.452] g[12.414]
>931, d1[0.264] d2[0.434] g[8.515]
>932, d1[0.265] d2[0.428] g[10.320]
>933, d1[0.254] d2[0.460] g[10.577]
>934, d1[0.262] d2[0.460] g[11.378]
>935, d1[0.259] d2[0.454] g[11.061]
>936, d1[0.259] d2[0.444] g[9.937]
>937, d1[0.257] d2[0.466] g[8.304]
>938, d1[0.242] d2[0.452] g[14.067]
>939, d1[0.264] d2[0.458] g[15.196]
>940, d1[0.269] d2[0.447] g[14.457]
>941, d1[0.259] d2[0.454] g[10.634]
>942, d1[0.256] d2[0.447] g[11.811]
>943, d1[0.248] d2[0.444] g[8.453]
>944, d1[0.271] d2[0.454] g[7.812]
>945, d1[0.244] d2[0.472] g[10.102]
>946, d1[0.256] d2[0.454] g[9.094]
>947, d1[0.258] d2[0.470] g[17.319]
>948, d1[0.237] d2[0.458] g[12.192]
>

...

>5609, d1[0.146] d2[0.721] g[11.851]
>5610, d1[0.129] d2[0.725] g[12.811]
>5611, d1[0.138] d2[0.793] g[8.168]
>5612, d1[0.129] d2[0.758] g[6.934]
>5613, d1[0.132] d2[0.751] g[9.542]
>5614, d1[0.132] d2[0.764] g[8.690]
>5615, d1[0.131] d2[0.752] g[9.223]
>5616, d1[0.137] d2[0.732] g[9.068]
>5617, d1[0.129] d2[0.745] g[13.454]
>5618, d1[0.124] d2[0.765] g[9.333]
>5619, d1[0.137] d2[0.753] g[11.306]
>5620, d1[0.142] d2[0.726] g[8.497]
>5621, d1[0.145] d2[0.732] g[9.117]
>5622, d1[0.129] d2[0.731] g[7.032]
>5623, d1[0.125] d2[0.735] g[10.299]
>5624, d1[0.122] d2[0.715] g[12.789]
>5625, d1[0.134] d2[0.790] g[9.337]
>5626, d1[0.137] d2[0.775] g[7.774]
>5627, d1[0.142] d2[0.730] g[9.478]
>5628, d1[0.133] d2[0.748] g[10.756]
>5629, d1[0.128] d2[0.746] g[9.406]
>5630, d1[0.141] d2[0.727] g[9.794]
>5631, d1[0.135] d2[0.761] g[13.164]
>5632, d1[0.129] d2[0.749] g[14.255]
>5633, d1[0.134] d2[0.792] g[8.944]
>5634, d1[0.133] d2[0.753] g[8.284]
>5635, d1[0.138] d2[0.764] g[14.047]
>5636, d1[0.139] d

>5835, d1[0.138] d2[0.766] g[12.524]
>5836, d1[0.138] d2[0.772] g[12.482]
>5837, d1[0.131] d2[0.744] g[10.201]
>5838, d1[0.121] d2[0.742] g[8.107]
>5839, d1[0.135] d2[0.763] g[10.241]
>5840, d1[0.130] d2[0.765] g[9.118]
>5841, d1[0.123] d2[0.744] g[10.901]
>5842, d1[0.120] d2[0.809] g[9.084]
>5843, d1[0.153] d2[0.759] g[7.639]
>5844, d1[0.118] d2[0.753] g[10.532]
>5845, d1[0.127] d2[0.785] g[9.522]
>5846, d1[0.135] d2[0.774] g[7.152]
>5847, d1[0.130] d2[0.766] g[6.757]
>5848, d1[0.129] d2[0.766] g[11.113]
>5849, d1[0.130] d2[0.757] g[8.563]
>5850, d1[0.143] d2[0.811] g[12.412]
>5851, d1[0.126] d2[0.766] g[8.817]
>5852, d1[0.134] d2[0.752] g[8.048]
>5853, d1[0.118] d2[0.724] g[12.030]
>5854, d1[0.127] d2[0.682] g[8.351]
>5855, d1[0.128] d2[0.750] g[10.175]
>5856, d1[0.123] d2[0.717] g[6.447]
>5857, d1[0.120] d2[0.732] g[7.329]
>5858, d1[0.141] d2[0.724] g[8.011]
>5859, d1[0.122] d2[0.759] g[11.238]
>5860, d1[0.129] d2[0.764] g[15.720]
>5861, d1[0.124] d2[0.755] g[9.112]
>5862, d1[0.123]

>6055, d1[0.124] d2[0.759] g[9.061]
>6056, d1[0.134] d2[0.747] g[7.631]
>6057, d1[0.128] d2[0.763] g[7.940]
>6058, d1[0.128] d2[0.745] g[10.377]
>6059, d1[0.136] d2[0.791] g[8.653]
>6060, d1[0.126] d2[0.763] g[9.854]
>6061, d1[0.131] d2[0.767] g[9.909]
>6062, d1[0.117] d2[0.712] g[13.117]
>6063, d1[0.141] d2[0.778] g[11.125]
>6064, d1[0.123] d2[0.795] g[8.580]
>6065, d1[0.131] d2[0.746] g[7.818]
>6066, d1[0.125] d2[0.773] g[9.197]
>6067, d1[0.125] d2[0.770] g[10.184]
>6068, d1[0.127] d2[0.752] g[11.576]
>6069, d1[0.134] d2[0.769] g[8.855]
>6070, d1[0.117] d2[0.718] g[12.437]
>6071, d1[0.143] d2[0.743] g[9.956]
>6072, d1[0.122] d2[0.711] g[10.388]
>6073, d1[0.122] d2[0.737] g[9.206]
>6074, d1[0.135] d2[0.766] g[10.280]
>6075, d1[0.124] d2[0.751] g[10.668]
>6076, d1[0.119] d2[0.721] g[12.252]
>6077, d1[0.124] d2[0.749] g[10.102]
>6078, d1[0.127] d2[0.771] g[7.627]
>6079, d1[0.136] d2[0.734] g[7.317]
>6080, d1[0.121] d2[0.786] g[8.400]
>6081, d1[0.124] d2[0.772] g[10.476]
>6082, d1[0.141]

>6280, d1[0.122] d2[0.764] g[11.856]
>6281, d1[0.127] d2[0.742] g[7.906]
>6282, d1[0.121] d2[0.739] g[9.510]
>6283, d1[0.147] d2[0.710] g[11.815]
>6284, d1[0.127] d2[0.737] g[9.613]
>6285, d1[0.150] d2[0.773] g[10.533]
>6286, d1[0.140] d2[0.753] g[9.113]
>6287, d1[0.129] d2[0.754] g[10.885]
>6288, d1[0.131] d2[0.755] g[10.429]
>6289, d1[0.144] d2[0.744] g[11.198]
>6290, d1[0.136] d2[0.778] g[7.872]
>6291, d1[0.138] d2[0.755] g[15.307]
>6292, d1[0.138] d2[0.722] g[7.357]
>6293, d1[0.143] d2[0.748] g[10.473]
>6294, d1[0.129] d2[0.776] g[8.236]
>6295, d1[0.119] d2[0.793] g[6.258]
>6296, d1[0.139] d2[0.754] g[10.632]
>6297, d1[0.121] d2[0.786] g[11.323]
>6298, d1[0.146] d2[0.724] g[9.591]
>6299, d1[0.132] d2[0.774] g[10.282]
>6300, d1[0.132] d2[0.741] g[10.030]
>6301, d1[0.126] d2[0.782] g[8.190]
>6302, d1[0.128] d2[0.697] g[9.671]
>6303, d1[0.141] d2[0.727] g[9.189]
>6304, d1[0.126] d2[0.766] g[10.462]
>6305, d1[0.127] d2[0.711] g[8.351]
>6306, d1[0.136] d2[0.770] g[6.670]
>6307, d1[0.136

KeyboardInterrupt: 

In [ ]:
#end the tensorflow session
from tensorflow.keras import backend as K
K.clear_session()

In [ ]:
#for cuda to release GPU memory
from numba import cuda
cuda.select_device(0)
cuda.close()